In [2]:
q = 2 
m = 6 
p = x^6 + x^4 + x^3 + x + 1
F.<a> = GF(q^m, modulus=p)
R.<x> = PolynomialRing(F) 

Suppose 
    m = 6 </br>
    a is prim. elem  </br>
    q = 2  </br>
    p(a) = a^6 + a^4 + a^3 + a + 1 


In [3]:
g = x^7 + x + 1
g_t = [1, 0, 0, 0, 0, 0, 1] # hardcoded for now: 
g_t

[1, 0, 0, 0, 0, 0, 1]

In [4]:
n = q^m 
T = g.degree() 
T

7

In [9]:
L =[e for e in F.list() if g(e) != 0 ]
L.append(L.pop(0))


In [10]:
# lookup_alpha
# L =[a^i for i in range(1, n+1) if g(a^i) != 0 ]
print(len(L))
Z = matrix(F, [[(1 / g(el)) if i == j else 0 for i, el in enumerate(L)] for j in range(n)])
Y = matrix(F, [[a_i^t for a_i in L] for t in range(T)])
X = [[0] * len(g_t) for _ in range(len(g_t))]
for i in range(len(g_t)): 
    for j in range(len(g_t)): 
        if j >= i: 
            X[j][i] = g_t[j-i]
X = matrix(F, X) 
print(f"dimension Z {Z.dimensions()}")
print(f"dimension Y {Y.dimensions()}")
print(f"dimension X {X.dimensions()}")

64
dimension Z (64, 64)
dimension Y (7, 64)
dimension X (7, 7)


In [11]:
H = X*Y*Z.T
H.dimensions()
H_ = [] 
for row in H.T: 
    vec = vector(F, list([vector(i) for i in row]))
    sss = [] 
    for r in vec: 
        sss += r 
    H_.append(sss) 
H_ = matrix(H_).T
H_.dimensions()

(42, 64)

In [12]:
generator_matrix = H_.right_kernel_matrix()
null_basis = generator_matrix*H.T 
if any(null_basis): 
    print("failed") 
else:
    print("worked!") 
generator_matrix.dimensions()

worked!


(22, 64)

Sanity check:

In [13]:
C = codes.GoppaCode(g, L) 

print(f"{C.minimum_distance()}") 
print(f"{C.distance_bound()}")
print(f"{C.generator_matrix().dimensions()}")
HHH = C.parity_check_matrix()
HHH.row_space() == H_.row_space()
C

15
8
(22, 64)


[64, 22] Goppa code over GF(2)

In [14]:
def text2bin(msg):
    return "".join(f"{ord(i):08b}" for i in msg) 
def bin2text(bin_s): 
    return "".join([chr(int(bin_s[i:i+8], 2)) for i in range(0, len(bin_s), 8)])
def to_field_element(string):
    """Binary String to field Element"""
    elem = 0
    n = len(string)
    for i in range(n):
        coeff = int(string[n-i-1])
        elem += coeff * a^i # coeff is 0 or 1
    return elem

def encodeGoppa(m):
    bin_m = text2bin(m) 
    m = m + "0"*(128 - len(bin_m)) 
    bin_m_chunks = [bin_m[i:i+64] for i in range(0, len(bin_m), 64)]
    half_blocks = []
    
    for half in bin_m_chunks:
        r_i = 0 
        for i, value in enumerate(half): 
            r_i += int(value) / (x - (a^i))
        if r_i.mod(g) == 0: # 
            half_blocks.append(r_i)
        else: 
            raise Exception("Was not zero!")
        r_i = 0 
    return half_blocks


In [15]:
generator_matrix

22 x 64 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

# Goppa decoding 

Pattersons algortihm for decoding. Corrects t design errors.  
Patterson converts a syndrome to a vector of erros. The syndrome of a binary word c = (c_0, ..., c_n-1) is 

In [16]:
# input to goppa decoding is a binary vector, of vector in base field 
def find_syndromes(received_vector):  
    codelocators = [] 
    F_2m = g.base_ring()
    PR_f_2m = g.parent() 
    syndrome_vector = vector(PR_f_2m, [(x - L[i]).inverse_mod(g) for i in range(len(L))]) 
    return syndrome_vector * received_vector

In [17]:
def from_list_to_poly(generic_list): 
    return sum([x for x in generic_list])

In [18]:
def _norm(a, b): 
    X = g.parent().gen()
    return 2^((a^2+X*b^2).degree())

def lattice_basis_reduction_for_pattersonAlg(AA, BB, t):
    
    old_vec = AA - (AA[0] // BB[0]) * BB
    
    if _norm(old_vec[0], old_vec[1]) > 2^t: 
        ALFA_BETA = vector((BB[1] % AA[0], (BB[0]//AA[0]) * old_vec[1]))
    else: 
        return old_vec
    
    while _norm(ALFA_BETA[0], ALFA_BETA[1]) > 2^t: 
        tmpAA = old_vec[0] % ALFA_BETA[0]
        tmpBB = old_vec[1] - (old_vec[0] // ALFA_BETA[0]) * ALFA_BETA[1]
        
        old_vec = ALFA_BETA
        
        ALFA_BETA = vector(tmpAA, tmpBB)
        
    return ALFA_BETA


In [19]:
def _split(p):
    # split polynomial p over F into even part po
    # and odd part p1 such that p(z) = p2 (z) + z p2 (z)
    Phi = p.parent()
    p0 = Phi([sqrt(c) for c in p.list()[0::2]]);
    p1 = Phi([sqrt(c) for c in p.list()[1::2]]);
    return (p0,p1);

def _g_inverse(p):
    (d, u, v) = xgcd(p, g)
    return u.mod(g)


def PattersonsDecodingForBinaryGoppaCodes(s_x, g_x): 
    if not any(s_x.list()): return vector([0])
    h = s_x.inverse_mod(g_x) 
    if h == x: 
        sigma_x = x 
        return sigma_x
    else: 
        #  
        (g0, g1) = _split(g_x)
        sqrt_X = g0 * _g_inverse(g1) 
        (T0, T1) = _split(_g_inverse(s_x) - x) 
        r_x = (T0 + sqrt_X * T1).mod(g_x)
        
        v1 = vector([r_x, 1])
        v2 = vector([g_x, 0])
        
        alpha, beta = lattice_basis_reduction_for_pattersonAlg(v2, v1, g_x.degree())
        
        sigma_x = alpha^2 + x * beta^2 
        return sigma_x

In [20]:
import random

def random_flip(lst, t):
    # Make a copy of the original list
    new_lst = list(lst)
    
    # Generate t random indices to flip
    indices = random.sample(range(len(lst)), t)
    
    # Flip the binary values at the selected indices
    for i in indices:
        new_lst[i] = 1 - lst[i]
    return new_lst


In [43]:
def decoding(r): 
    # assume r is on vector/binary form 
    s_x = find_syndromes(r) 
    if not any(s_x.list()): return r 
    sigma = PattersonsDecodingForBinaryGoppaCodes(s_x, g) 
    positions = sigma.roots() 
    # fix errors 
    error_values_pos = [] 
    for p, e in positions: 
        if p != 0:  
            error_values_pos.append(p.log(a)) 
        else: 
            error_values_pos.append(0)
    
    r = list(r)
    for corr in error_values_pos: 
        r[corr] += 1
    
    return vector(GF(2), r)#  * generator_matrix.T

NUM_ERR = 1 # up to t errors

message = vector(F, [random.choice([0, 1]) for _ in range(22)])
cword = vector(F, message * generator_matrix)
format_cword = ', '.join([str(x) for x in cword])
print(f"Original message: {format_cword}")

print()
# apply noise 
rword = vector(F, random_flip(cword.list(), 3))
format_rword = ', '.join([str(x) for x in rword])
print(f"Received vec: {format_rword}")


print() 
for i, e in enumerate(rword.list()): 
    cword_ = cword.list() 
    if e != cword_[i]: 
        print(f"added error at: {i} \n")

# corrected_word = decoding(rword) 
# print(f"Decoding sucess: {corrected_word == cword}")
# str_format = ', '.join([str(x) for x in corrected_word.list()]) 
# print(f"Correct word was: {str_format}")


correct_word = decoding(rword)
print
print(f"Decoding Sucess: {(correct_word) == cword}")
str_format = ', '.join([str(x) for x in correct_word.list()])
print(f"Correct word was: {str_format}")

Original message: 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0

Received vec: 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0

added error at: 26 

added error at: 37 

added error at: 54 

Decoding Sucess: False
Correct word was: 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0


## Key Generation

In [22]:


# Create private key & public key 
k = 22 
text = "TIMEFLIESQUICKLY" 
code_word = encodeGoppa(text)

In [23]:
# Scrambler matrix S 
# S = matrix(GF(2), k, [random.random() < 0.5 for _ in range(k^2)])
# while (rank(S) < k): 
#    S[floor(k * random.random()), floor(k*random.random())] += 1
# S

In [265]:
def find_S():
    # Generate a random binary matrix of size k x k
    S = matrix(GF(2), k, [random.random() < 0.5 for _ in range(k^2)])

    # Ensure the matrix has full rank (k)
    while rank(S) < k:
        # Choose a random row and column
        row = floor(k * random.random())
        col = floor(k * random.random())
        # Add 1 to the chosen element
        S[row, col] += 1
    return S

In [266]:
S = find_S()
S

22 x 22 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

In [281]:
# Permutation matrix P 
def find_P():
    n = q^m
    P = identity_matrix(GF(2),n) 
    columns = list(range(n)) 
    for i in range(n): 
        col_idx = columns.pop(randrange(len(columns)))
        row_idx = randrange(i, n) 
        P[row_idx, col_idx] = 1 
    
    if P.is_invertible(): 
        return matrix(P)
    else: 
        return find_P()

P = find_P()

In [282]:
P.is_invertible()

True

In [283]:
# y = mG' + e 
# G' = SGP 
G_prime = S * generator_matrix * P

In [284]:
# cipher_text = Encryption(message)# 

In [297]:
def rowVectorWeight(n): 
    weight = 0 
    for i in range(n.ncols()):
        if n[0, i] == 1: 
            weight = weight + 1
    return weight

def Encryption(message):
    assert len(message) == G_prime.nrows()
    # Get an error vec that is t errors 
    t = 7 
    err_vec = matrix(1, generator_matrix.ncols())
    while rowVectorWeight(err_vec) < 7:
        err_vec[0, random.randint(1, generator_matrix.ncols()-1)] = 1   
    cipher = message * G_prime
    return cipher + vector(err_vec.list())

# cipher_text = Encryption(message)
# len(cipher_text) # 64 - nice ! 

In [303]:
message = vector(random.choice([0, 1]) for _ in range(22))
message

(1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1)

In [304]:
ciphertext = Encryption(message) 

In [305]:
def decode(ciphertext):
    ciphertext = ciphertext * P
    plaintext = decoding(ciphertext)
    return generator_matrix.solve_right(message)[:22]

In [306]:
print(decode(ciphertext))

(1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0)


In [359]:
bin_m = text2bin(text)  # text to binary 
bin_m = bin_m + '0'* (22 - (len(bin_m) % 22)) # added 12 padding to the last block 
blocks = [vector(GF(2), bin_m[i:i+22]) for i in range(0, len(bin_m), 22)]

# encryption
encrypted_blocks = []
for block in blocks: 
    encrypt_block = Encryption(block)
    encrypted_blocks.append(block)
    
print(encrypted_blocks[0])

(0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1)


In [360]:
# decryption
plaintext_blocks = []
for block in encrypted_blocks: 
    plaintext = decode(block) 
    plaintext_blocks.append(plaintext)

TypeError: unsupported operand parent(s) for *: 'Vector space of dimension 22 over Finite Field of size 2' and 'Full MatrixSpace of 64 by 64 dense matrices over Finite Field of size 2'

In [361]:
# convert to plaintext 
for msg_block in plaintext_blocks: 
    print(bin2text(''.join(list(map(str, msg_block.list())))))